####  columns =['NASDAQ','S&P','CBOE','ExchangeRate','futures2y','futures10y']

In [1]:
!pip install selenium

In [4]:
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time

In [5]:
content = []

# 2년 만기 미국채 선물

In [15]:
# webdirver 설정

driver = webdriver.Chrome('C:\chromedriver.exe')
driver.get('https://kr.investing.com/rates-bonds/us-2-yr-t-note-historical-data')
driver.set_window_size(1080,1080)

In [16]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [17]:
us2yr = soup.select('#curr_table > tbody > tr:nth-child(1) > td:nth-child(2)')
us2yr[0].text.strip()
content.append(us2yr[0].text.strip())

In [18]:
del soup
driver.close()

# 10년 만기 미국채 선물

In [19]:
driver = webdriver.Chrome('C:\chromedriver.exe')
driver.get('https://kr.investing.com/rates-bonds/us-10-yr-t-note-historical-data')
driver.set_window_size(1080,1080)

In [20]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [21]:
us10yr = soup.select('#curr_table > tbody > tr:nth-child(1) > td:nth-child(2)')
us10yr[0].text.strip()
content.append(us10yr[0].text.strip())

In [22]:
del soup
driver.close()

### 날짜 인덱스 부여

In [23]:
content

['109.98', '131.53']

In [72]:
from datetime import date, datetime, timedelta

#date = datetime.today().replace(hour=0, minute=0, second=0)
#date = pd.DatetimeIndex([date.strftime('%Y-%m-%d %H:%M:%S')])
date = pd.DatetimeIndex([datetime.today().date()])
date

DatetimeIndex(['2021-09-28'], dtype='datetime64[ns]', freq=None)

In [73]:
import pandas as pd

df = pd.DataFrame([content], date, ['futures2y','futures10y'])
df

,futures2y,futures10y
2021-09-28,109.98,131.53


In [75]:
df = df.reset_index().rename(columns={'index': 'date'})
df

,date,futures2y,futures10y
0,2021-09-28,109.98,131.53


In [71]:
df_crawl = df_new.set_index('date')
df_crawl

,futures2y,futures10y
date,,
2021-09-28,109.98,131.53


# 함수화

In [62]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from datetime import date, datetime, timedelta


date

class INDEXCRAWL:
    def __init__(self):
        self.contents = []
    
    def crawling(self,url):
        self.driver = webdriver.Chrome('C:\chromedriver.exe')
        self.driver.get(url)
        self.driver.set_window_size(720,720) 
        #self.driver.execute_script("window.scrollTo(0, 700)")
        
        self.soup = BeautifulSoup(self.driver.page_source, 'html.parser')
        self.index = self.soup.select('#last_last')
        #self.index = self.soup.select('#curr_table > tbody > tr:nth-child(1) > td:nth-child(2)')
        self.contents.append(self.index[0].text.strip())
        
        del self.soup
        self.driver.quit()
        
        
    def saving(self):
        self.date = pd.DatetimeIndex([datetime.today().date()])
        self.df = pd.DataFrame([self.contents], self.date, ['futures2y','futures10y'])
        print(self.df)
        
    

if __name__ == "__main__":
    crawl = INDEXCRAWL()
    us2yr ='https://kr.investing.com/rates-bonds/us-2-yr-t-note-historical-data'
    crawl.crawling(us2yr)
    us10yr = 'https://kr.investing.com/rates-bonds/us-10-yr-t-note-historical-data'
    crawl.crawling(us10yr)
    crawl.saving()


           futures2y futures10y
2021-09-28    109.97     131.42
